<a href="https://colab.research.google.com/github/Darkdew/Web-scrapper-jsload-dynamic/blob/main/Australia_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
url = 'https://www.medhelp.org/forums/STDs/show/116?page=1'

r = requests.get(url, headers= headers)
soup = BeautifulSoup(r.text, 'html.parser')
page_count = int(16920/6)
page_count
full_list=[]
for page_cnt in range(1,page_count):
        
        #generic page url
        main_url= 'https://www.medhelp.org/forums/STDs/show/116?page='

        #concatenating page number upto 2820
        current_url = main_url+ str(page_cnt)
        print(current_url)
        
        #runnign soup on page=n
        r0 = requests.get(current_url, headers= headers)
        soup0 = BeautifulSoup(r0.text, 'html.parser')
        
        #counting number of available threads on page=n
        threads = soup0.find_all('div',{'class':'username'})
        number_of_threads= len(threads)




        # r_current_url = requests.get(current_url, headers= headers)
        # current_url_soup = BeautifulSoup(r_current_url.text, 'html.parser')
        # current_url_thread_name = current_url_soup.find_all('div',{'class':'avt avt-sm'})
              
        for count in range(number_of_threads):
            temp_list=[]
            # Extracting thread title
            thread_title= soup0.find_all('div',{'class':'subj_header'})[count].find_all('a')[1].text

            #Extracting Username
            username= soup0.find_all('div',{'class':'username'})[count].find('a').text
            print(username)
            #Extracting_profile_url
            profile_extension = soup0.find_all('div',{'class':'username'})[count].find('a')['href']
            profile_url= 'https://www.medhelp.org' + profile_extension

            r1 = requests.get(profile_url, headers= headers)
            soup1 = BeautifulSoup(r1.text, 'html.parser')

            print(profile_url)
            #profile_details
            profile_details= soup1.find_all('div',{'class':'section'})
            breaking= str(profile_details[0]).split('</span>')

            #AboutMe
            try:
              AboutMe= breaking[1].replace('<span>','').replace(',','')

            except Exception as e:
              AboutMe= 'NA'
            #print(AboutMe)

            #membership Date
            try:
              Membership_date= breaking[2].replace('\n','').replace('</div>','').replace(',','')
            #print(Membership_date)
            except Exception as e:
              Membership_date=breaking[0]
            
            #part_of_communities
            try:
              communities= soup1.find('div',{'id':'my_comm'}).find_all('a')
              community_list=[]
          
              for i in range(2,55,3):
                try:
                  community= communities[i].text
                except Exception as e:
                  break
                community_list.append(community)
          
            except Exception as e:
              community_list='NA'
            print(community_list)

            temp_list=[thread_title, username, profile_url,AboutMe,Membership_date,community_list]
            full_list.append(temp_list)
  
df = pd.DataFrame(full_list,columns=("Thread_Title","Username","Profile_url","AboutMe","Membership_date","is part of Communities"))
df.to_csv('/content/drive/MyDrive/Colab Data/master_df_aus1.csv', index= False)

df.head()

df.to_csv('/content/drive/MyDrive/Colab Data/master_df_aus_2.csv', index=False)
import pandas as pd
full_final_list=[]
for index, row in df.iterrows():
    temp_list1 =[]
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'}
    
    profile_url= row['Profile_url']
    print(profile_url)

    #crawling through profile page
    r2=wd.get(profile_url)
    r3=wd.page_source#)  # results
    soup2 = BeautifulSoup(r3, 'html.parser')

    #See-All posts url
    all_posts_url_value= soup2.find('div',{'id': 'mw_subtitle_2'})
    try: 
      posts_url= all_posts_url_value.find('a')['href']
      see_all_posts_url= 'https://www.medhelp.org'+posts_url
      print(see_all_posts_url)
    
    except:
      continue
    #crawling on all posts page
    r4=wd.get(see_all_posts_url)
    r5=wd.page_source#)  # results
    soup3 = BeautifulSoup(r5, 'html.parser')

    #counting posts on page
    response_link= soup3.find_all('div',{'class':'subj_user os_12'})

    #looping over the available posts
    for j in range(len(response_link)):
      #capturing response_url
      response_link_url= response_link[j].find('a')['href']
      response_link_url_final= 'https://www.medhelp.org'+response_link_url
      print(response_link_url_final)
      
      #crawling on specific_response_url
      r6=wd.get(response_link_url_final)
      r7=wd.page_source#)  # results

      soup4 = BeautifulSoup(r7, 'html.parser')

      ##
      resp_locn= response_link_url_final.split('post_')[1]
      div_class= 'mh_vit_resp_ctn post_ctn_'+ resp_locn

      #response_text_final
      
      #len(response_text)
      try:
        response_text= soup4.find_all('div',{'class':div_class})
        #print(response_text)
        response_text_final= response_text[0].find('div',{'class':'resp_body '}).text
      except Exception as e:
        response_text_final= soup4.find_all('div',{'id':'subject_msg'})[0].text
      ##response_time_final
      try:
        response_time_final= response_text[0].find('time',{'class':'mh_timestamp'}).text

      except Exception as e:
        response_time_final=soup4.find_all('div',{'class':'username'})[0].find('time')["datetime"]
      temp_list1= [row['Thread_Title'], row['Username'], row['Profile_url'], row['AboutMe'], row['Membership_date'], row['is part of Communities'], response_text_final, response_time_final]
      full_final_list.append(temp_list1)

import pandas as pd

df_final = pd.DataFrame(full_final_list,columns=("Thread_Title","Username","Profile_url","AboutMe","Membership_date","is part of Communities","Response_text","Response_time"))
df_final.to_csv('/content/drive/MyDrive/Colab Data/df_aus_final_final.csv', index=False)